In [ ]:
import pandas as pd
import numpy as np  
from lib.question_type_recommender import question_type_recommender
from docx.api import Document

from lib.picklist_recommender import picklist_recommender
from lib.issue_parser import issue_parser 
from lib.questionnaire_parser import questionnaire_parser
from lib.dbupdate_parser import dbupdate_parser
from lib.script_generator import script_generator 
from lib.db_parser import db_parser  

import re, json, logging, requests, random 
from lib.context import context
from lib.utils import *   
from lib.script_generator import script_generator 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
qr = question_type_recommender(ctx, verbose=True, use_cache=True)
plr=picklist_recommender(ctx, use_cache=False) 
db_parser = db_parser(ctx)

In [ ]:
qr.recommend('How many systems does Anthony need for the system')

In [ ]:
qt = sql_todf("SELECT CAST(PK_QuestionTypeId AS NVARCHAR(99)) as [{PK_QuestionType}], code AS [{QT_CODE}] FROM fsma_QuestionTypes WHERE code IN ('LABEL','YN','CNT','TXT','PICK','FREQ','DEC','MULTICHECKBOX')",  ctx.connstr) 
qt

HVA PARSE


In [ ]:
df24 = db_parser.parse(question_group='5140:5146') 
df24['{PK_PickListType}'] = df24['{PK_PickListType}'].apply(lambda s: str(s).replace('None','')).astype(str) 
df24['{PK_Question24}'] = df24['{PK_Question}'].apply(lambda s: int(s)).astype(str)  
df24['{PK_QuestionGroup24}'] = df24['{PK_QuestionGroup}'].apply(lambda s: int(s)).astype(str) 
df24['{sortpos24}'] = df24['{sortpos}'].apply(lambda s: int(s)) 
df24['{FK_QuestionType}'] = df24['{FK_QuestionType}'].astype(str) 
df24 = df24[['{IdText}', '{FK_QuestionType}', '{PK_Question24}', '{PK_QuestionGroup24}', '{sortpos24}']]
df24 = df24.sort_values(by=['{PK_QuestionGroup24}', '{sortpos24}'])
df24 

In [ ]:
df24.loc[ df24['{IdText}'] == '7.0']

In [ ]:

def hvaprocess( df25 ):
    df25 = df25.fillna('') 
    df25 = df25.astype('string')
    df25 = df25.loc[ df25['{FY25}'] != 'Deleted' ]  
    df25['{FY24}'] = [ q[1] if re.match('.*New.*', q[0] ) else q[0] for q in zip( df25['{FY24}'], df25['{FY25}'] )  ]  
    df25['{QuestionText}'] =  df25['{QuestionText}'].apply(lambda s: re.sub('\n|\r',r'', s) )
    df25['{ro}'] =  df25['{ro}'].apply(lambda s: re.sub('\n|\r',r'', s) )
    df25['{change}'] =  df25['{change}'].apply(lambda s: re.sub('\n|\r',r'', s) )
    df25['{QuestionText}'] =  df25['{QuestionText}'].apply(lambda s: s.replace("'","''")) 
    df25['{QuestionText}'] = [ q[0] if re.match('.*[a-z].*', q[0] ) else q[1] for q in zip( df25['{FY24}'], df25['{QuestionText}'] )  ]   
    df25 = df25[ df25['{QuestionText}'] != '' ]  
    
    df25['{sortpos}'] = df25.index + 1 
    df25['{PK_QuestionType}'] = '9' 
    df25['{dep}'] = '' 
    df25['{FK_Question_Master}'] = ''
    df25['{PK_PickListType}'] = 'NULL'
    for i,r in df25.iterrows():     
        fmt = r['{fmt}']
        m = re.match('(.*):(.*)', fmt, re.DOTALL) 
        if m: 
            pl = [p.strip() for p in re.sub('\n\n','\n',m.groups(0)[1]).split('\n')  if p.strip() != ''  ] 
            rec = plr.recommend(pl, generate_sql=False, threshhold=(.825, .7))  
            df25.loc[i, '{PK_PickListType}']=str(rec['PK_PicklistType'])

        df25.loc[i, '{PK_QuestionType}']=qr.recommend(r['{QuestionText}'])['PK_QuestionType'].astype(str) 
        dep = re.match('.*If.*(\s[\.|\d]+)\s?.*', r['{QuestionText}'] ) 
        if(dep): 
            depid =dep.groups(0)[0] 
            df25.loc[i, '{dep}'] = depid.strip()
      
    df25['{Dependancy}'] = [f'depend="{q[0]}"' if re.match('.*\d.*',q[0])  else '' for q in zip(df25['{dep}'], df25['{PK_QuestionType}'])]

    df25['{PK_QuestionType}'] = ['17' if re.match('.*Picklist.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['17' if re.match('.*Select One.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])] 
    df25['{PK_QuestionType}'] = ['43' if re.match('.*Select.*Apply.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['2' if re.match('.*Integer.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['1' if re.match('.*Yes.*No.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['18' if re.match('.*Indicator.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_PickListType}'] = ['NULL' if re.match('.*Yes.*No.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_PickListType}'])]
    df25['{PK_PickListType}']=df25['{PK_PickListType}'].fillna('')
    df25['{ro}'] = df25['{ro}'].apply(lambda s: re.sub('\n|\r','',s))
    return df25

def gen_scripts(source, seed):

    file = fr'{ctx.get_download_path()}\hva\{source}.xlsx' 
    df25 = pd.read_excel(file, header=0, sheet_name='Sheet1',  usecols='A:Z', dtype=str)  
     
    df25 = hvaprocess(df25)
    df25['{PK_Question}'] = (df25['{sortpos}']+seed).astype(str)  

    df25['{FY24}'] = df25['{FY24}'].apply(lambda s: str(s) + '.0'  if '.' not in s else s)  
    df25['{FY25}'] = df25['{FY25}'].apply(lambda s: str(s) + '.0'  if '.' not in s else s) 
    if re.match('1B', file): 
        df25['{FY24}'] = df25['{FY24}'].apply(lambda s: re.sub('B(\d)',r"B \1", str(s)) ) 
        df25['{FY24}'] = df25['{FY24}'].apply(lambda s: s if re.match('^1B.*', s) else '0')  
        df25['{FY25}'] = [ q[0] if re.match('^0.*', q[0] ) else q[1] for q in zip( df25['{FY24}'], df25['{FY25}'] )  ]
    df25['{idt}'] = df25['{FY25}']
    df25['{FY24}'] = [ q[1] if re.match('.*New.*', q[0], re.IGNORECASE ) else q[0] for q in zip( df25['{FY24}'], df25['{FY25}'] )  ]
    df25['{PK_QuestionType}'] = ['18' if re.match('^\.?0',q[0])  else q[1] for q in zip(df25['{FY25}'], df25['{PK_QuestionType}'])]
    df25 = pd.merge(df25, qt, how='left', left_on='{PK_QuestionType}', right_on='{PK_QuestionType}')
    df25 = df25.astype('string')
    print(file)
    if re.match('.*45.*', file): 
        df25['{PK_QuestionType}'] = '92'
        
    gen=script_generator(ctx)
    code=gen.generate( df25 , code_template_path=f'{ctx.get_tempalte_dir()}hva_insert.sql')   
    with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
        f.write( f'\n{code}' )  
    df25['{idt}'] = df25['{idt}'].apply(lambda s: s.replace('.','_'))
    df25['{FK_PickListType}'] = df25['{PK_PickListType}'] 
    
    form_code = gen.generate(df25)  
    with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
        form_code =   form_code.replace('\ufeff', '')
        fw.write(form_code)
    return df25

In [ ]:
d={'1A':72000, '1B':72125, 'S2':72245, 'S3': 72270, '45':72320}
df25 = gen_scripts('S3' , 72270)
df25[0:8] 

In [ ]:
d={'1A':72000, '1B':72125, 'S2':72245, 'S3': 72270, '45':72320}
df25 = gen_scripts('S3' , 72270)
df25[0:8] 

In [ ]:
def get_dep(s):
    m=re.match('.*\s(\d\d?\.[\d\.|\.]+)\s.*', s)
    if m:
        return m.groups(0)[0]
    m=re.match('If\s.*[Yes].*', s.strip())
    if m:
        return 'y'
    m=re.match('If\s.*[No].*', s.strip())
    if m:
        return 'n'
    return
get_dep("If 'Yes' the the quesiton")

In [ ]:
dff25 = db_parser.parse(question_group='5490:5491')[['{IdText}','{PK_Question}','{QuestionText}', '{PK_PickListType}']] 
dff25['{PK_Question}']=dff25['{PK_Question}'].astype(str)
dff25['depID'] = dff25['{QuestionText}'].apply( get_dep )
dff25['depID'] = [re.sub('(\d{,2})\..*',r'\1.0', str(q[1]))  if q[0] =='y' else q[0] for q in zip(dff25['depID'], dff25['{IdText}'])]
dff25[18:] 
merged = pd.merge(dff25,dff25,how='left', left_on='depID', right_on='{IdText}', suffixes=('','_dep')) 
# merged['depID'] = [re.sub('(\d{,2})\..*',r'\1.0', str(q[1]))  if q[0] =='y' else q[0] for q in zip(merged['depID'], merged['{IdText}'])]
merged[22:27] 


In [ ]:
pl = picklist_recommender(ctx, use_cache=False)
for i, r in df2425.iterrows(): 
    if 'picklist' in r['{change}'].lower(): 
        lst = re.sub('Select all that Apply:\s*\n|Select One:\s*\n', '', r['{fmt}'], flags=re.IGNORECASE) 
        lst=lst.split('\n')  
        pl.recommend(lst, generate_sql=True, threshhold=(1.825, 1.7))
        pl

In [ ]:
qr = question_type_recommender( ctx, verbose=True, use_cache=True)

In [ ]:
picklist_string = 'Date Determined / TBD / Continuous ATO'  
pl=picklist_recommender(ctx, use_cache=False).recommend(picklist_string.split(' / '), generate_sql=True, threshhold=(.825, .7))
pl 

In [ ]:
config = {}
with open('C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\AutomatorConfig.json', 'rb') as f: 
    config=json.loads(f.read())    
dic = config['InputDefaults']["//form[contains(@action, '_CIO_')]"]

lod=[]
for k in dic.keys():
    lod.append({'regex':k, 'value':dic[k]})
df=pd.DataFrame(lod)
df.to_csv( r"C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\form[contains(@action, '_CIO_')].csv" , index=False)


In [ ]:
pk = sql_todf("SELECT DisplayValue, PK_Picklist FROM vwPicklists WHERE UsageField='ServiceModelType' ", config['connstr'])
pk

In [ ]:
ep = sql_todf("""
SELECT
	  REPLACE(p.name,'cq_','') AS [spname]
    , pa.parameter_id AS [order]
    , pa.name AS [name]
    , UPPER(t.name) AS [type]
    , t.max_length AS [length]  
FROM sys.parameters AS pa 
INNER JOIN sys.procedures AS p on pa.object_id = p.object_id 
INNER JOIN sys.types AS t on pa.system_type_id = t.system_type_id AND pa.user_type_id = t.user_type_id
WHERE 1=1
AND create_date > '06-06-2024'
AND p.name LIKE  'cq_%'

""", config['connstr'])
ep



In [ ]:
ep_g = ep[['spname','name']].groupby('spname', as_index=False).agg({'name':list})
ep_g

In [ ]:
def to_routes(lst):
    s= map( lambda s: '[FromQuery] string? '+s.replace('@','')+', ' , lst) 
    return ''.join(list(s))
to_routes(['@PK_ReportingCycle','@ComponentGroup'])


In [ ]:
def to_params(lst): 
    rq,p='','' 
    for l in lst:
        l= re.sub('@','',l)
        #rq=rq+f'\n    var {l} = request.Query["{l}"] ?? ""; '
        p=p+f'\n    request.PARMS.Add("@{l}", {l}); '
    return rq+''+p
s = to_params(['@Generic1','@PK_ReportingCycle'])
print(s)

In [ ]:
snip=''
with open(ctx.get_tempalte_dir()+ 'snip.cs', 'r', encoding='UTF-8') as f: 
    snip=f.read() 
s=''
for i,r in ep_g.iterrows():
    args = to_routes(r['name'])
    #print(params) 
    s=s+snip.replace('~args~', args).replace('~cq~', r['spname']).replace('~parms~', to_params(r['name']))

print(s)

with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{s}' ) 

In [ ]:
dest=''
with open(r'C:\dev\CyberScope\CsLab\Cyberscope\CSwebdev\code\CSAPI\Program.cs', 'r', encoding='UTF-8') as f: 
    dest=f.read() 
m=re.search('.*(MapGet.+)app\.Run',dest, flags=re.DOTALL)
m


In [ ]:
from bs4 import BeautifulSoup
soup = ''
f="C:/Users/timko/Downloads/IG25.htm"
with open(f, encoding='unicode_escape') as file:
  soup = BeautifulSoup(file, 'html.parser')
links = soup.select('a')
dct = {}
for t in links: 
  if t.select('span'):
    ltext = re.sub('\n|\r',' ', str(t.select('span')[0].text)  )
    ltext=ltext.replace('  ','')
    if ltext not in dct.keys() and (re.sub('\s','',ltext) != '' ):
      href=t["href"]
      href='"'+ href +'"'
      dct[ltext]=f"<a href={href} target=_blank>{ltext}</a>"
dct 


In [64]:
sql = ''
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_7_DBUpdate7.70_IG.sql') as file:
  sql=file.read()
for k in dct.keys(): 
  sql=sql.replace(k, dct[k])
with open(r'C:\temp\CyberScopeBranch_Release\_7_DBUpdate7.70.1_IG.sql', 'w+') as file:
  file.write(sql)


In [ ]:

links = ''.join(list(links)).replace('<span style="font-size:8.0pt">','')
links





In [ ]:
[{"MAX_PK_ReportingCycle":185
,"MAX_PK_QuestionGroup":5495
,"MAX_PK_FormPage":4995
,"MAX_PK_Question":72341
,"_2024AIG_PK_Question":"63800:63961"
,"_2024AIG_FK_FormPage":"4621:4631"
,"_2024AIG_PK_QuestionGroup":"5121:5131"
,"_2024AIG_PK_ReportingCycle":"162"}]

In [ ]:
document = Document(r"C:\Users\timko\Downloads\IG25.docx")
tables = document.tables 
row=document.tables[1].rows[2]
cell=row.cells[2]
cell
   

In [ ]:
document = Document(r"C:\Users\timko\Downloads\IG25.docx")
cnt = 1
fmt = ", (@PK_FormPage+1, @PK_Form,'title', 'FismaForms/2024/2024_A_IG_0.aspx', 1)"
# fmt = ", (@PK_QGroup+1,'group','group: title',@PK_Form, @PK_FormPage+1, 1)"
for paragraph in document.paragraphs:
    match = re.match('Table \d{1,2}:(.*)',paragraph.text)
    if(match):
        title=match.groups(0)[0].strip()
        txt = fmt.replace('title',title)
        txt = txt.replace('1',str(cnt))
        txt = txt.replace('0',str(cnt))
        print(txt)
        cnt+=1
 

In [ ]:
sections =  ['5', '7', '14', '17', '21', '24', '26', '30', '33']

In [ ]:
tables = document.tables 
row=document.tables[4].rows[1]
cols=[cell.text for cell in row.cells] 

In [ ]:
tbl = []
for i in range(len(document.tables)):
    for r in range(2, len(document.tables[i].rows)):
        row = document.tables[i].rows[r]
        match=re.match('\d{1,2}', row.cells[0].text)
        if match:
            text = [cell.text for cell in row.cells]
            tbl.append(text[:9]) 
tbl[:2]

In [ ]:
df = pd.DataFrame(  data=tbl, columns=cols ) 
df['{group}'] = 0 
df['{sortpos}'] = df.index+1  
df = df.replace('’', '`', regex=True)
df = df.replace("'", '`', regex=True) 
df = df.replace('\n', '<br>', regex=True)
df['{idt}'] = df['Question'].apply(lambda s: re.match('(\d{1,3}.?\d?).+', s).groups(0)[0])
df['{QuestionText}'] = df['Question']
df  = df.drop(columns=['Review\nCycle', 'Question'], axis=1 )

df['{PK_Question}'] = 72400
df['{help}'] = '<h5>Maturity Level Definitions</h5><hr><h6>Ad Hoc</h6>'+ df['Ad Hoc'] + '<h6>Defined</h6>'+ df['Defined'] + '<h6>Consistently Implemented</h6>'+ df['Consistently Implemented'] + '<h6>Managed and Measurable</h6>'+ df['Managed and Measurable'] + '<h6>Optimized</h6>'+ df['Optimized'] +'<hr>'+ df['Supplemental Guidance'] +'<hr>'+ df['Criteria']     
 
pk = 72400
sec = 1
spos = 1
for i,r in df.iterrows():
    if r['{idt}'].replace('.','') in sections:
        sec+=1
        pk+=5
        spos+=2
    df.at[i, '{sortpos}'] = spos + i
    df.at[i, '{group}'] = sec
    df.at[i, '{PK_Question}'] = pk + i
 
    if 'positive or negative' in df.at[i, '{QuestionText}'] :
        df.at[i, '{help}'] = ''
    if 'Effective/Not Effective' in df.at[i, '{QuestionText}'] :
        df.at[i, '{help}'] = ''

        
dff = df.iloc[:, 7:13] 

dff.to_csv(ctx.get_dest()+'script.csv', index=False)
dff.to_html(ctx.get_dest()+'script.html', index=False)
dff = df.iloc[:, 7:13] 
dff['ins']="INSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup,sortpos,FK_PickListType, FK_QuestionType,identifier_text,ExternalLinkType, QuestionText, help_text) VALUES 	"
dff[:3]



In [ ]:
gen=script_generator(ctx)
sql = " ({PK_Question},@FormName, @PK_QGroup+{group},{sortpos},85,17,N'{idt}',NULL,N'{QuestionText}',N'{help}')"
lst = [] 
for i,r in dff.iterrows():
    code=gen.generate(dff.iloc[i:i+1], code_template_path=f'{ctx.get_tempalte_dir()}ig.sql')
    code=code.replace('\n','')
    code=code.replace(".',", "',") 

    if 'Effective/' in code:
        code=code.replace(',85',',528') 
        
    if 'additional information' in code:
        code=code.replace(',17',',9')
        code=code.replace('85,','NULL,')
        
    if str(r['{idt}']).replace('.','') in sections or r['{idt}']=='1.':
        print('\n' + r['ins'])
        print( code.replace(', (', '  ('))
    else: 
        print( code)
 
 

In [ ]:
gen=script_generator(ctx)
code=gen.generate(dff, code_template_path=f'{ctx.get_tempalte_dir()}ig.sql')  


In [ ]:
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' ) 

In [ ]:
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'r', encoding='UTF-8') as f: 
    sql=f.read() 
m=re.search('--PARSEBEGIN(.*)--PARSEEND',sql, flags=re.DOTALL)
sql=sql.replace(m.group(1),code)
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 

#with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'w', encoding='UTF-8') as f: 
#    f.write() 